In [2]:
import numpy as np
import pandas as pd
import os
import sys as sys
import socket
import matplotlib.pyplot as plt
import multiprocessing as mp
import itertools as it
#from adjustText import adjust_text

import warnings
warnings.filterwarnings("ignore") # Great Style
socket_name = socket.gethostname()
print(socket_name)

if socket_name.startswith("compute-"):
    print("HSM Computational partition detected.")
    path = "/n/groups/reich/hringbauer/git/punic_aDNA/"  # The Path on Midway Cluster
else:
    raise RuntimeWarning("No compatible machine. Check!!")

os.chdir(path)  # Set the right Path (in line with Atom default)
# Show the current working directory. Should be HAPSBURG/Notebooks/ParallelRuns

print(os.getcwd())
print(f"CPU Count: {mp.cpu_count()}")

#sys.path.insert(0,"/n/groups/reich/hringbauer/git/hapBLOCK/package/")  # hack to get development package first in path
from ancIBD.ibd_stats.funcs import new_columns, find_relatives, give_sub_df, plot_age_diff, rc_date

compute-e-16-229.o2.rc.hms.harvard.edu
HSM Computational partition detected.
/n/groups/reich/hringbauer/git/punic_aDNA
CPU Count: 28


In [6]:
path_save = f"/n/groups/reich/hringbauer/git/punic_aDNA/output/ibd/v54.1.ibd_ind.d220.tsv" # for IBD csv file
df = pd.read_csv(path_save, sep="\t")

path_save = "/n/groups/reich/hringbauer/Data/v54.1.anno.csv"# for IBD csv file
df_meta = pd.read_csv(path_save, sep=",")

new_columns(df, df_meta, col = ["age", "loc", "n_cov_snp"]) #"clst"

df=df.sort_values(by="sum_IBD>20", ascending=False)
df20 = df[df["max_IBD"]>16].copy()

In [7]:
df20

,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2
0,I35351,I35333,282.987110,3332.207697,21.0,3332.207697,21.0,3332.207697,21.0,3332.207697,21.0,2500.0,2500.0,"Cap Bon, Kelibia","Cap Bon, Kelibia",1028493,986415
1,I24494,I24040,283.652203,3326.638294,20.0,3326.638294,20.0,3326.638294,20.0,3326.638294,20.0,2464.0,2547.0,Kerkouene,Kerkouene,791516,702598
2,I22122,I22118,213.983188,3308.844301,25.0,3308.844301,25.0,3308.844301,25.0,3274.698507,23.0,2419.0,2589.0,"Sardinia, Tharros","Sardinia, Tharros",711166,802359
3,I24494,I24193,283.652203,2048.808393,33.0,2029.570302,31.0,2001.710003,29.0,1966.079616,27.0,2464.0,2500.0,Kerkouene,Kerkouene,791516,789887
4,I22122,I22117,176.386502,1706.853619,24.0,1706.853619,24.0,1706.853619,24.0,1706.853619,24.0,2419.0,2582.0,"Sardinia, Tharros","Sardinia, Tharros",711166,791179
5,I24193,I24040,106.422907,980.703286,33.0,933.513776,28.0,893.512584,25.0,805.672194,20.0,2500.0,2547.0,Kerkouene,Kerkouene,789887,702598
7,I24494,I24194,75.874001,713.637284,16.0,695.799482,14.0,695.799482,14.0,678.831780,13.0,2464.0,2500.0,Kerkouene,Kerkouene,791516,791265
6,I24194,I24193,77.233100,728.634680,23.0,710.130383,21.0,670.256979,18.0,653.336687,17.0,2500.0,2500.0,Kerkouene,Kerkouene,791265,789887
9,I24494,I24215,89.881498,473.034386,10.0,473.034386,10.0,473.034386,10.0,456.281487,9.0,2464.0,2505.0,Kerkouene,Kerkouene,791516,707101
8,I18201,I18199,47.389899,650.968208,36.0,515.483997,22.0,417.833296,15.0,350.126997,11.0,2554.0,2226.0,"Almería, Cuevas del Almazora, Villaricos","Almería, Cuevas del Almazora, Villaricos",881415,744521


In [9]:
give_sub_df(df, pop1='Carthage', pop2='', col='loc')

Found: 48 Pairs fitting in dataframe.



,iid1,iid2,max_IBD,sum_IBD>8,n_IBD>8,sum_IBD>12,n_IBD>12,sum_IBD>16,n_IBD>16,sum_IBD>20,n_IBD>20,age1,age2,loc1,loc2,n_cov_snp1,n_cov_snp2
24,I35336,I35338,36.030801,36.030801,1.0,36.030801,1.0,36.030801,1.0,36.030801,1.0,2500.0,2500.0,"Tunis, Carthage","Tunis, Carthage",609059,627665
34,I12847,I28504,23.704901,23.704901,1.0,23.704901,1.0,23.704901,1.0,23.704901,1.0,2591.0,2337.0,"Sicily, Birgi",Carthage,771978,679772
183,I12666,I35336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2455.0,2500.0,"Sicily, Birgi","Tunis, Carthage",821361,609059
188,I12846,I28504,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2093.0,2337.0,"Sicily, Marsala, Lilybaeum, Tribunale",Carthage,829186,679772
148,I35351,I35336,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2500.0,2500.0,"Cap Bon, Kelibia","Tunis, Carthage",1028493,609059
151,I35335,I35338,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2500.0,2500.0,"Cap Bon, Kelibia","Tunis, Carthage",1073568,627665
153,I4799,I28504,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2442.0,2337.0,"Sicily, Motya",Carthage,837043,679772
154,I24678,I35325,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2599.0,2500.0,"Sicily, Birgi","Tunis, Carthage",682219,803743
191,I21853,I35339,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2464.0,2500.0,"Sicily, Selinunte, Manuzza","Tunis, Carthage",761629,759862
220,I24555,I35325,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.000000,0.0,2250.0,2500.0,"Sicily, Birgi","Tunis, Carthage",841605,803743
